In [1]:
import random
import pandas as pd
import numpy as np
import itertools

In [2]:
df = pd.read_csv('./FS_SCR_DB/FS_SCR_DB.TXT')


In [10]:
import pandas as pd
import numpy as np
import random

def find_optimal_distribution_random(df_sorted, initial_percentiles, default_rate_ranges, num_seeds, gumo, upjong):
    results = []

    for seed in range(num_seeds):
        random.seed(seed)  # 시드 고정
        new_percentiles = initial_percentiles.copy()

        for i in range(len(new_percentiles)):
            # ±2% 범위 내에서 랜덤 숫자 생성 (소수점 2자리)
            adjustment = round(random.uniform(-2, 2), 2)
            if new_percentiles[i] == 0 :
                new_percentiles[i] == 0
            else :
                new_percentiles[i] += adjustment
                new_percentiles[i] = max(0, new_percentiles[i])  # 음수 방지

        # 현재 비율 합 계산
        total_percent = sum(new_percentiles)

        # 전체 비율을 100%로 재조정
        if total_percent > 0:  # 혹시라도 0이 되는 경우 방지
            new_percentiles = [(x / total_percent) * 100 for x in new_percentiles]

        # 비율 합이 100이 아닌 경우 스킵 (오차 방지)
        if abs(sum(new_percentiles) - 100) > 1e-9:
            continue


        # count 기준으로 등급 경계 계산
        total_count = df_sorted['count'].sum()
        boundaries = (np.cumsum(new_percentiles) / 100 * total_count).astype(int)

        df_sorted['cumsum'] = df_sorted['count'].cumsum()
        df_sorted['Grade'] = np.nan

        start_idx = 0
        for grade, boundary in enumerate(boundaries, start=1):
            df_sorted.loc[(df_sorted['cumsum'] > start_idx) & (df_sorted['cumsum'] <= boundary), 'Grade'] = grade
            start_idx = boundary

        df_sorted.loc[df_sorted['Grade'].isna(), 'Grade'] = 9

        # 등급별 부도율 계산
        grade_summary = df_sorted.groupby("Grade").agg(
            total_count=("count", "sum"),
            BUDO_TAG_sum=("BUDO_TAG_sum", "sum")
        )
        grade_summary["default_rate"] = grade_summary["BUDO_TAG_sum"] / grade_summary["total_count"] * 100

        # 부도율 역전 조건 확인
        is_reversed = np.all(np.diff(grade_summary["default_rate"]) >= 0)

        # 부도율 범위 초과 여부 확인
        is_valid = all(grade_summary["default_rate"].get(i + 1, 0) <= default_rate_ranges[i + 1] for i in range(len(default_rate_ranges)))

        # 등급별 비율 차이 계산
        difference = sum(abs(new_percentiles - initial_percentiles))

        # 결과 저장
        results.append({
            "GUMO": gumo,
            "UPJONG": upjong,
            **{f"{i+1}등급": new_percentiles[i] for i in range(len(new_percentiles))},
            **{f"{i+1}등급_부도율": grade_summary["default_rate"].get(i+1, 0) for i in range(len(default_rate_ranges))},
            "부도율_만족": is_valid,
            "역전_만족": is_reversed,
            "비율차이합": difference
        })

    df_results = pd.DataFrame(results)
    
    # 조건 우선순위 적용 및 상위 10개 선택
    df_filtered = df_results[df_results['부도율_만족'] & df_results['역전_만족']]
    if df_filtered.empty:
        df_filtered = df_results[df_results['역전_만족']]
    if df_filtered.empty:
        df_filtered = df_results[df_results['부도율_만족']]
    if df_filtered.empty:
        df_filtered = df_results  # 모든 조건이 만족되지 않으면 전체 데이터 사용

    df_final = df_filtered.nsmallest(10, "비율차이합")

    # 순위 컬럼 추가
    df_final['순위'] = df_final['비율차이합'].rank()

    return df_final


def process_and_save_results(df, gumo_range, upjong_range, initial_percentiles_df, default_rate_ranges, num_seeds):
    all_results = []  # 모든 결과를 하나의 리스트에 모음

    for gumo in gumo_range:
        for upjong in upjong_range:
            df1 = df[(df["GUMO"] == gumo) & (df['UPJONG'] == upjong)]
            df2 = df1.groupby('SCORE').agg(
                count=('SCORE', 'size'),
                BUDO_TAG_sum=('BUDO_TAG', 'sum')
            ).reset_index()

            # SCORE 기준으로 내림차순 정렬
            df_sorted = df2.sort_values(by="SCORE", ascending=False).reset_index(drop=True)

            # initial_percentiles 데이터프레임에서 해당 GUMO와 UPJONG 값 찾기
            initial_percentiles_row = initial_percentiles_df[(initial_percentiles_df['GUMO'] == gumo) & (initial_percentiles_df['UPJONG'] == upjong)]
            if initial_percentiles_row.empty:
                continue
            initial_percentiles = (initial_percentiles_row.iloc[0, 2:].values.astype(float) * 100)

            df_results = find_optimal_distribution_random(df_sorted, initial_percentiles, default_rate_ranges, num_seeds, gumo, upjong)

            # 모든 결과를 리스트에 추가
            all_results.append(df_results)

    # 모든 결과를 하나의 데이터프레임으로 결합
    final_results = pd.concat(all_results, ignore_index=True)

    # 결과를 하나의 Excel 파일에 저장
    filename = "./result/result.xlsx"
    final_results.to_excel(filename, index=False)
    print(f"Saved all results to {filename}")

# 파라미터 설정
gumo_range = range(5, 9)  # 1~8
upjong_range = range(1, 8)  # 1~7
initial_percentiles_df = pd.read_excel('./등급계량화_기준값_20250212.xlsx')

default_rate_ranges = {1: 0.1, 2: 0.2, 3: 0.3, 4: 0.7, 5: 1.6, 6: 3.1, 7: 5.4, 8: 8.9, 9: 100}
num_seeds = 20000

# 데이터프레임(df) 필요 -> 실행 전에 로드 필요

In [11]:
process_and_save_results(df, gumo_range, upjong_range, initial_percentiles_df, default_rate_ranges, num_seeds)

Saved all results to ./result/result.xlsx
